In [1]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

## 데이터 준비

In [2]:
origin=read_excel('행정동병합테스트22(개발제한행정동제외).xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,2154,437,337.55,4,0.352898,0,32,0,3,4552.529183
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,1808,267,60.79,0,0.063554,0,27,0,12,7611.382114
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,626,110,956.51,3,1.000000,0,16,0,5,1728.859060
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,1814,311,0.00,3,0.000000,10,34,0,5,4130.837004
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,3495,491,1.14,3,0.001192,10,54,0,5,1990.755355


In [3]:
origin.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도'],
      dtype='object')

In [4]:
origin.isna().sum()
origin1=origin.fillna(0)

## 결측치는 0으로 처리

> 실제 조사된 값이므로 없다면 관측되지 아니함으로 판단

In [5]:
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도'],
      dtype='object')

## 데이터 스케일링

In [6]:
scaler=MinMaxScaler()
df = origin1.copy()
df['인구minmax']=scaler.fit_transform(df[['계(명)']])
df['지하철minmax']=scaler.fit_transform(df[['지하철역_수']])
df['주차장면수minmax']=scaler.fit_transform(df[['면수 (공영+민영)']])
df['버스정류장minmax']=scaler.fit_transform(df[['행정동별버스정류장수']])
df['자전거대여소minmax']=scaler.fit_transform(df[['공공자전거대여소수']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,인구minmax,지하철minmax,주차장면수minmax,버스정류장minmax,자전거대여소minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0,32,0,3,4552.529183,0.221482,0.000000,0.083273,0.294118,0.065217
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,27,0,12,7611.382114,0.176957,0.285714,0.234960,0.245098,0.260870
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0,16,0,5,1728.859060,0.047724,0.000000,0.038693,0.137255,0.108696
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,10,34,0,5,4130.837004,0.177242,0.000000,0.063226,0.313725,0.108696
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,10,54,0,5,1990.755355,0.334946,0.000000,0.174100,0.509804,0.108696


## 파생변수(`접근성지수`) 생성

In [7]:
df['접근성지수'] = (df['지하철minmax'] + df['주차장면수minmax'] + df['버스정류장minmax'] + df['자전거대여소minmax']) / df['인구minmax']
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,인구밀도,인구minmax,지하철minmax,주차장면수minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,32,0,3,4552.529183,0.221482,0.000000,0.083273,0.294118,0.065217,1.998398
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,27,0,12,7611.382114,0.176957,0.285714,0.234960,0.245098,0.260870,5.801654
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,16,0,5,1728.859060,0.047724,0.000000,0.038693,0.137255,0.108696,5.964340
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,34,0,5,4130.837004,0.177242,0.000000,0.063226,0.313725,0.108696,2.740019
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,54,0,5,1990.755355,0.334946,0.000000,0.174100,0.509804,0.108696,2.366352


## 파생 변수의 값 정렬

In [8]:
df['접근성지수'].value_counts().sort_index()

접근성지수
0.408163     1
0.472881     1
0.486704     1
0.542889     1
0.570193     1
            ..
12.580041    1
14.126776    1
14.478314    1
17.408026    1
inf          1
Name: count, Length: 406, dtype: int64

## 결측치 확인

In [9]:
df['접근성지수'].isna().sum()

0

In [10]:
df['접근성지수'].unique()

array([ 1.99839798,  5.80165397,  5.96434022,  2.74001876,  2.36635189,
        0.8701663 ,  2.00762348,  4.64623944, 14.47831372,  6.89286884,
        3.96437435,  1.76480448,  2.8053561 ,  0.95438794,  2.31354388,
        2.34776302,  1.01134175, 14.12677625, 12.58004128, 17.4080256 ,
        5.86746654,  4.50315443,  6.2462551 , 12.45516291,  4.09511653,
        1.26516904,  1.87496695,  0.48670359,  0.76748728,  0.80102911,
        0.84436466,  1.41552893,  0.99800828,  2.49224131,  6.25150415,
        1.19522991,  1.5043489 ,  1.91284072,  2.54970472,  1.21444708,
        5.54458544,  1.05550237,  0.98429708,  2.92135285,  1.26266493,
        3.02879589,  2.56372929,  0.68233624,  0.91206592,  1.26841822,
        1.66741749,  2.92878413,  3.55520746,  1.30827907,  1.96135017,
        1.7688502 ,  1.70476624,  2.13820245,  1.88960002,  2.56661767,
        1.85191328,  1.62891204,  3.9535831 ,  1.39542169,  2.70900459,
        1.48880851,  1.44557536,  2.0746163 ,  0.72043616,  1.53

In [11]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '인구밀도', '인구minmax', '지하철minmax', '주차장면수minmax',
       '버스정류장minmax', '자전거대여소minmax', '접근성지수'],
      dtype='object')

## 지수 파일 생성에 사용될 컬럼 추출

In [12]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','계(명)','지하철역_수','면수 (공영+민영)','행정동별버스정류장수', '공공자전거대여소수', '인구minmax','지하철minmax','주차장면수minmax','버스정류장minmax', '자전거대여소minmax', '접근성지수'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,계(명),지하철역_수,면수 (공영+민영),행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장면수minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,11110,종로구,1111051500,청운효자동,11700,0,4592,32,3,0.221482,0.000000,0.083273,0.294118,0.065217,1.998398
1,11110,종로구,1111053000,사직동,9362,2,12605,27,12,0.176957,0.285714,0.234960,0.245098,0.260870,5.801654
2,11110,종로구,1111054000,삼청동,2576,0,2237,16,5,0.047724,0.000000,0.038693,0.137255,0.108696,5.964340
3,11110,종로구,1111055000,부암동,9377,0,3533,34,5,0.177242,0.000000,0.063226,0.313725,0.108696,2.740019
4,11110,종로구,1111056000,평창동,17658,0,9390,54,5,0.334946,0.000000,0.174100,0.509804,0.108696,2.366352


## 지수 파일 생성

In [13]:
df1.to_excel('접근성지수1.xlsx', index=False)

In [14]:
df2=read_excel('접근성지수1.xlsx')
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,계(명),지하철역_수,면수 (공영+민영),행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장면수minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,11110,종로구,1111051500,청운효자동,11700,0,4592,32,3,0.221482,0.000000,0.083273,0.294118,0.065217,1.998398
1,11110,종로구,1111053000,사직동,9362,2,12605,27,12,0.176957,0.285714,0.234960,0.245098,0.260870,5.801654
2,11110,종로구,1111054000,삼청동,2576,0,2237,16,5,0.047724,0.000000,0.038693,0.137255,0.108696,5.964340
3,11110,종로구,1111055000,부암동,9377,0,3533,34,5,0.177242,0.000000,0.063226,0.313725,0.108696,2.740019
4,11110,종로구,1111056000,평창동,17658,0,9390,54,5,0.334946,0.000000,0.174100,0.509804,0.108696,2.366352


## 이상치 `inf`, `-inf` 처리

In [15]:
# DataFrame(df2)에서 '접근성지수' 열을 선택
accessibility_series = df2['접근성지수']

# 'inf'와 '-inf'를 0으로 대체
accessibility_series = accessibility_series.replace([np.inf, -np.inf], 0)

# 변경된 '접근성지수' 열을 DataFrame에 할당
df2['접근성지수'] = accessibility_series

In [16]:
df2.to_excel("접근성지수1.xlsx", index=False)

In [17]:
df2 = read_excel("접근성지수1.xlsx")
df2.head()

,시군구코드,시군구명,행정동코드,읍면동명,계(명),지하철역_수,면수 (공영+민영),행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장면수minmax,버스정류장minmax,자전거대여소minmax,접근성지수
0,11110,종로구,1111051500,청운효자동,11700,0,4592,32,3,0.221482,0.000000,0.083273,0.294118,0.065217,1.998398
1,11110,종로구,1111053000,사직동,9362,2,12605,27,12,0.176957,0.285714,0.234960,0.245098,0.260870,5.801654
2,11110,종로구,1111054000,삼청동,2576,0,2237,16,5,0.047724,0.000000,0.038693,0.137255,0.108696,5.964340
3,11110,종로구,1111055000,부암동,9377,0,3533,34,5,0.177242,0.000000,0.063226,0.313725,0.108696,2.740019
4,11110,종로구,1111056000,평창동,17658,0,9390,54,5,0.334946,0.000000,0.174100,0.509804,0.108696,2.366352


## 지수 정규화

In [18]:
df2['접근성지수minmax'] = scaler.fit_transform(df2[['접근성지수']])
df2

,시군구코드,시군구명,행정동코드,읍면동명,계(명),지하철역_수,면수 (공영+민영),행정동별버스정류장수,공공자전거대여소수,인구minmax,지하철minmax,주차장면수minmax,버스정류장minmax,자전거대여소minmax,접근성지수,접근성지수minmax
0,11110,종로구,1111051500,청운효자동,11700,0,4592,32,3,0.221482,0.000000,0.083273,0.294118,0.065217,1.998398,0.114798
1,11110,종로구,1111053000,사직동,9362,2,12605,27,12,0.176957,0.285714,0.234960,0.245098,0.260870,5.801654,0.333275
2,11110,종로구,1111054000,삼청동,2576,0,2237,16,5,0.047724,0.000000,0.038693,0.137255,0.108696,5.964340,0.342620
3,11110,종로구,1111055000,부암동,9377,0,3533,34,5,0.177242,0.000000,0.063226,0.313725,0.108696,2.740019,0.157400
4,11110,종로구,1111056000,평창동,17658,0,9390,54,5,0.334946,0.000000,0.174100,0.509804,0.108696,2.366352,0.135935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,11740,강동구,1174065000,성내제2동,23303,3,7869,18,5,0.442449,0.428571,0.145307,0.156863,0.108696,1.897251,0.108987
402,11740,강동구,1174066000,성내제3동,22882,1,10656,15,6,0.434432,0.142857,0.198065,0.127451,0.130435,1.378372,0.079180
403,11740,강동구,1174068500,길동,45142,1,17661,36,7,0.858351,0.142857,0.330671,0.333333,0.152174,1.117299,0.064183
404,11740,강동구,1174069000,둔촌제1동,70,1,193,9,4,0.000000,0.142857,0.000000,0.068627,0.086957,0.000000,0.000000


## 최종 데이터 저장

In [21]:
df2.to_excel('접근성지수1.xlsx',index=False)